# Using the KPoEM Dataset and Model: Poetry Generation

- **KPpEM Emoiton Classification Medel** : AKS-DHLAB. (2025). KPoEM  [Computer software]. Hugging Face. https://doi.org/10.57967/hf/6301 
- This code is uploaded in AKS-DHLAB. (2025). KPoEM [Computer software]. GitHub. https://github.com/AKS-DHLAB/KPoEM  

## 1. Basic setup and library imports
- Import Libraries and Set Configuration

In [1]:
!pip install -q huggingface_hub langchain langchain-core langchain-community langchain-huggingface 
!pip install faiss-cpu

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/bitsandbytes-0.45.4.dev0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_thunder-0.2.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/nvfuser-0.2.23a0+6627725-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utiliti

In [2]:
# 라이브러리 임포트
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, ElectraModel, AutoModelForCausalLM, pipeline
from huggingface_hub import hf_hub_download
import os
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import FAISS

## 2. Loading the KPoEM Model

In [3]:
# 기초 세팅
REPO_ID = "AKS-DHLAB/KPoEM" # 허깅페이스에 업로드된 감정분류모델 id
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") #GPU 사용
THRESH_HOLD = 0.3

In [4]:
# KPoEM_Classifier 클래스
class KPoEM_Classifier(nn.Module):
    def __init__(self, repo_id, device):
        self.labels = [
            '불평/불만', '환영/호의', '감동/감탄', '지긋지긋', '고마움', '슬픔', '화남/분노', '존경',
            '기대감', '우쭐댐/무시함', '안타까움/실망', '비장함', '의심/불신', '뿌듯함', '편안/쾌적',
            '신기함/관심', '아껴주는', '부끄러움', '공포/무서움', '절망', '한심함', '역겨움/징그러움',
            '짜증', '어이없음', '없음', '패배/자기혐오', '귀찮음', '힘듦/지침', '즐거움/신남', '깨달음',
            '죄책감', '증오/혐오', '흐뭇함(귀여움/예쁨)', '당황/난처', '경악', '부담/안_내킴', '서러움',
            '재미없음', '불쌍함/연민', '놀람', '행복', '불안/걱정', '기쁨', '안심/신뢰'
        ]
        num_labels = len(self.labels)
        #모델 & 토크나이저 로드
        super().__init__()
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(repo_id) 
        self.electra = AutoModel.from_pretrained(repo_id)
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(self.electra.config.hidden_size, num_labels)
        )

        weights_path = hf_hub_download(repo_id=repo_id, filename="classifier_state.bin") #가중치 불러오기
        self.classifier.load_state_dict(torch.load(weights_path, map_location=self.device))
        self.to(self.device)
        self.eval()

    # 텍스트를 입력받아 최종 logits 반환
    def forward(self, text: str):
        encoding = self.tokenizer(
          text,
          add_special_tokens=True,
          max_length=512,
          padding="max_length",
          truncation=True,
          return_tensors='pt',
        ).to(self.device)

        with torch.no_grad():
            outputs = self.electra(
                input_ids=encoding["input_ids"],
                attention_mask=encoding["attention_mask"],
                token_type_ids=encoding["token_type_ids"]
            )

        pooled_output = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(pooled_output)
        return logits

    def analyze(self, text: str, threshold=0):
        logits = self.forward(text)
        probabilities = torch.sigmoid(logits.squeeze()) #확률로 변환 → threshold 이상이면 선택
        predictions = (probabilities > threshold).int()

        result_dict = {
            self.labels[i]: float(round(probabilities[i].item(), 3))
            for i, label_id in enumerate(predictions)
            if label_id == 1
        }

        # 확률값 기준 내림차순 정렬된 dict로 반환
        result_dict = dict(sorted(result_dict.items(), key=lambda x: x[1], reverse=True))
        return result_dict


In [5]:
# KPoEM 모델 로드
print(f"... '{DEVICE}' 환경에서 '{REPO_ID}' 모델을 로드하고 있습니다 ...")
kpoem_model = KPoEM_Classifier(repo_id=REPO_ID, device=DEVICE)
print("KPoEM 모델을 성공적으로 로드하였습니다.")

... 'cpu' 환경에서 'AKS-DHLAB/KPoEM' 모델을 로드하고 있습니다 ...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/823 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/434M [00:00<?, ?B/s]

classifier_state.bin:   0%|          | 0.00/137k [00:00<?, ?B/s]

KPoEM 모델을 성공적으로 로드하였습니다.


In [27]:
# 모델 사용 테스트
test = """미풍에 웃는 아침을 기원하련다"""
result = kpoem_model.analyze(test, threshold=THRESH_HOLD)
result

{'기대감': 0.919,
 '기쁨': 0.809,
 '즐거움/신남': 0.635,
 '행복': 0.622,
 '비장함': 0.511,
 '환영/호의': 0.496,
 '아껴주는': 0.372,
 '편안/쾌적': 0.32,
 '감동/감탄': 0.304}

## 3. Downloading and Loading the LLM for Poetry Generation

In [7]:
MODEL_ID = "K-intelligence/Midm-2.0-Base-Instruct"
CACHE_DIR = "/home/work/KPoEM/code/AICreation/model"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

#모델 로드 - device_map="auto" 유지 (자동 분산 로드)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map="auto",           # GPU + CPU 메모리 자동 분산
    low_cpu_mem_usage=True,      # 로딩 시 CPU 메모리 절약
    trust_remote_code=True,
    cache_dir=CACHE_DIR
)

print("모델 로드 완료. GPU/CPU 자동 오프로딩 활성화")

# HuggingFace pipeline 생성 (device 설정하지 않아도 자동)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    temperature=0.7,
    top_p=0.9,
    max_new_tokens=512,
    repetition_penalty=1.2
)

# LangChain LLM 래퍼로 감싸기
llm = HuggingFacePipeline(pipeline=pipe)

# 간단한 프롬프트 템플릿
prompt = PromptTemplate(
    input_variables=["topic"],
    template="다음 주제로 감성적인 시를 써주세요:\n주제: {topic}\n\n### 시:\n"
)

# LLMChain 생성
chain = LLMChain(llm=llm, prompt=prompt)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/10.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Device set to use cpu


모델 로드 완료. GPU/CPU 자동 오프로딩 활성화


/tmp/ipykernel_1847/3750988198.py:31: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
/tmp/ipykernel_1847/3750988198.py:40: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


## 4. Prompt-Based Poetry Generation 1
Using Input Text and Emotion Classification Results (Without a Vector Database)

### 감정 분류 모델만 적용하여 LLM으로 시 생성(Vector DB 미적용)

In [8]:
def extract_poetry_section(template):
    # Split the template by "### 시:" and extract the part after it
    if "### 시:" in template:
        poetry_section = template.split("### 시:")[1].strip()
        # Split by lines and return as a list
        poetry_lines = poetry_section.splitlines()
        return poetry_lines
    else:
        return None

In [9]:
# 5️⃣ 전체 흐름 함수
def emotion_to_poetry(sample_text): #감정 분류에 사용할 사용자 텍스트 인풋.
    emotion_scores = kpoem_model.analyze(sample_text, threshold=0.3)
    
    template = """
    ### 시스템:
    당신은 창의적이고 감성적인 근현대 시인입니다.
    아래에 제시된 원문 텍스트와 감정 목록을 참고하여 시를 지으세요.
    원문 텍스트는 시의 소재나 분위기를 떠올리는 데 활용하고,
    감정 목록에 언급된 감정들을 시의 핵심 정서로 반영하세요.
    
    영어나 다른 언어는 사용하지 말고, 한국어로만 작성하세요.
    이모지나 그림은 사용하지 마세요.
    한국 고유의 표현을 사용하고,
    은유와 상징을 통해 창의적으로 감정을 표현하세요.
    시 해설은 필요 없습니다. 시만 작성하세요.
    
    ### 원문 텍스트:
    {sample_text}
    
    ### 감정 목록:
    {emotion}
    
    ### 시:
    """

    # """ ### 시스템: 당신은 창의적이고 감성적인 근현대 시인입니다. 다음 감정목록에 언급된 감정들을 주된 시의 정서로 활용하세요. 영어나 다른 언어는 사용하지 말고, 한국어로만 작성하세요. 이모지나 그림을 사용하지 마세요. {context_snippets} 위 문장들에서 옛스러운 한국 고유의 표현을 사용하여 시를 하나 지어주세요. 은유와 상징을 사용하여 창의적으로 감정을 표현하세요. 시 해설은 필요없습니다. 시만 써주세요. 감정 목록: {top_emotion} ### 시: """

    prompt = PromptTemplate(
        input_variables=["emotion"],
        template=template.strip()
    )
    # print(emotion_scores)
    chain = LLMChain(llm=llm, prompt=prompt)
    result = chain.run(sample_text=sample_text, emotion=emotion_scores)
    
    return result

In [10]:
sample_text = """미풍에 웃는 아침을 기원하련다"""

In [13]:
# # 6️⃣ 테스트
generated_poem = emotion_to_poetry(sample_text)

In [14]:
print("생성된 시:\n")
extract_poetry_section(generated_poem)

생성된 시:



['아침 창가에 앉은 그대',
 '미풍이 살며시 불어오는 소리 들으며',
 '웃고 있는 내 마음 속 작은 꽃 하나',
 '',
 '어제보다 더 밝은 오늘이여',
 '그대가 내게 준 기쁨처럼',
 '햇살도 덩달아 춤추네',
 '',
 '꽃잎 사이 스며드는 바람결마다',
 '설레임 가득한 기다림 있어라',
 '희망이라는 이름의 푸른 날개짓',
 '',
 '내일이면 또 만날 수 있겠지',
 '그리운 눈빛 마주하며 웃을 그날까지',
 '가슴속 깊이 새긴 약속',
 '',
 '그대 향한 이 마음 끝닿도록',
 '영원히 이어질 우리 이야기여',
 '바람 따라 흐르는 행복 노래하리라']

## 5. Prompt-Based Poetry Generation 2
Using Input Text and Emotion Classification Results (With a Vector Database)

#### - Loading the Vector Store Built with the KcELECTRA Backbone Model

In [15]:
# KcELECTRA 기반 커스텀 임베딩 클래스 정의
class KcELECTRAEmbeddings(Embeddings):
    def __init__(self, model_name: str = "beomi/KcELECTRA-base", device: str = "cpu"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(device)
        self.device = device

    def _embed(self, text: str):
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :]
        return cls_embedding.squeeze().cpu().numpy()

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        return [self._embed(text).tolist() for text in texts]

    def embed_query(self, text: str) -> list[float]:
        return self._embed(text).tolist()

In [16]:
# 3️⃣ 벡터 임베딩 모델 로딩 (한국어 지원하는 모델 권장) KcElectra -> backbone 모델로 사용
embedding_model = KcELECTRAEmbeddings()

tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/514 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [18]:
# 로컬에서 로드 (신뢰할 수 있는 파일일 경우)
vectorstore = FAISS.load_local(
    "./data/poetry_vectorstore", #깃허브 파일은 링크 수정 "./vectorstore" 참고 - https://github.com/AKS-DHLAB/KPoEM/blob/main/KPoEM_vectorstore.ipynb
    embedding_model, 
    allow_dangerous_deserialization=True
)

#### - After Computing Vector Similarity, 
Use the Emotion Information Stored in the Metadata of the Retrieved 100 Entries to Construct Context for Poetry Generation

In [19]:
class PoetryGenerator:
    """감정 분석 + 문맥 검색 + 시 생성까지 전체 파이프라인을 수행하는 클래스"""
    
    def __init__(self, kpoem_model, vectorstore, llm):
        """
        클래스 초기화
        Args:
            kpoem_model: KPoEM 감정 분류 모델
            vectorstore: FAISS 등 벡터스토어 객체
            llm: LangChain LLM 객체
        """
        self.kpoem_model = kpoem_model
        self.vectorstore = vectorstore
        self.llm = llm
    # 상위 감정 리스트 반환
    def get_top_emotions(self, scores, top_n=10):
        """점수 dict에서 상위 N개의 감정을 선택하고 key 리스트 반환"""
        top_emotions = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_n])
        return top_emotions, list(top_emotions.keys())
    # 상위 감정과 벡터DB의 메타데이터의 "emotion"값을 비교하여 가장 유사한 시구 10개를 선택
    def filter_context_by_emotion(self, context, top_emotion_keys, top_k=10):
        """context(Document 리스트)에서 감정 교집합 개수로 필터링 및 정렬."""
        results_with_score = []
        for doc in context:
            doc_emotions = set(doc.metadata.get("emotion", {}).keys())
            overlap = doc_emotions.intersection(top_emotion_keys)
            if overlap:
                results_with_score.append((doc, len(overlap), overlap))
        # 교집합 개수 많은 순으로 정렬 후 상위 top_k 선택
        results_with_score.sort(key=lambda x: x[1], reverse=True)
        return [doc for doc, _, _ in results_with_score[:top_k]]

    def build_prompt_template(self):
        """시 생성 프롬프트 템플릿 생성"""
        return PromptTemplate(
            input_variables=["top_emotion", "context_snippets"],
            template="""
            ### 시스템:
            당신은 창의적이고 감성적인 근현대 시인입니다.  
            아래에 제시된 원문 텍스트와 감정 목록을 참고하여 시를 지으세요.
            원문 텍스트는 시의 소재나 분위기를 떠올리는 데 활용하고,
            감정 목록에 언급된 감정들을 시의 핵심 정서로 반영하세요.  
            영어나 다른 언어는 사용하지 말고, 한국어로만 작성하세요.  
            이모지나 그림을 사용하지 마세요.  

            아래에 제시된 참고 문장들에서 한국 고유의 표현을 사용하고,
            은유와 상징을 통해 창의적으로 감정을 표현하세요.
            시 해설은 필요 없습니다. 시만 작성하세요.
            **원문 텍스트:**  
            {sample_text}  

            **참고 문장들:**  
            {context_snippets}  

            위 문장들을 바탕으로, 옛스러운 한국 고유의 표현을 사용하여 시를 하나 지어주세요.  
            은유와 상징을 사용하여 창의적으로 감정을 표현하세요.  
            시 해설은 필요없습니다. 시만 써주세요.  

            **감정 목록:** {top_emotion}  

            ### 시:""".strip()
        )

    def generate_poetry(self, user_input, top_n=10, context_k=100, filtered_k=10):
        """전체 파이프라인 실행: 감정 분석 → 문맥 검색 → 시 생성"""
        # 1. 감정 분석
        scores = self.kpoem_model.analyze(user_input, threshold=0.3) #인풋 텍스트 감정분류
        top_emotions, top_emotion_keys = self.get_top_emotions(scores, top_n=top_n) #상위 감정 라벨 가져오기

        # 2. 유사 문맥 검색
        context = self.vectorstore.similarity_search(user_input, k=context_k) # 인풋 텍스트에 대한 벡터 유사도가 근접한 VecotreDB에 저장된 데이터 호출(100개로 세팅)
        filtered_results = self.filter_context_by_emotion(context, top_emotion_keys, top_k=filtered_k) # 감정 값도 비슷한 백터유사도 근점 시구 필터링
        print("검색된 감정유사한 벡터DB:", filtered_results)
        # 3. 문맥 텍스트 생성
        context_text = "\n".join([doc.page_content for doc in filtered_results]) # 프롬프트엔지니어링에 사용할 문맥 텍스트 생성(RAG)
        # print(context_text)
        # 4. 프롬프트 생성 및 LLM 실행
        prompt = self.build_prompt_template()
        chain = LLMChain(llm=self.llm, prompt=prompt)
        result = chain.run(sample_text=sample_text, top_emotion=top_emotions, context_snippets=context_text)

        return result


In [20]:
# 초기화
generator = PoetryGenerator(kpoem_model=kpoem_model, vectorstore=vectorstore, llm=llm)

In [21]:
# 테스트 실행
user_text = """미풍에 웃는 아침을 기원하련다"""

In [22]:
# 시 생성
poem = generator.generate_poetry(user_text, top_n=10, context_k=100, filtered_k=10)
print(poem)

검색된 감정유사한 벡터DB: [Document(id='e1bc9080-46f6-4be9-9a91-e06a5d53f800', metadata={'emotion': {'기대감': 1.0, '기쁨': 1.0, '깨달음': 0.6, '감동/감탄': 0.4, '행복': 0.4, '환영/호의': 0.2, '비장함': 0.2, '뿌듯함': 0.2, '편안/쾌적': 0.2, '아껴주는': 0.2, '즐거움/신남': 0.2}, 'poet': '김소월'}, page_content='그러하다, 봄날은 꿈꿀 때.'), Document(id='7c6e549f-a4f7-4a36-886d-f7a825875bbc', metadata={'emotion': {'기대감': 0.8, '뿌듯함': 0.6, '아껴주는': 0.6, '행복': 0.6, '기쁨': 0.6, '존경': 0.4, '즐거움/신남': 0.4, '흐뭇함(귀여움/예쁨)': 0.4, '안심/신뢰': 0.4, '환영/호의': 0.2, '감동/감탄': 0.2, '고마움': 0.2, '우쭐댐/무시함': 0.2, '비장함': 0.2, '깨달음': 0.2}, 'poet': '한용운'}, page_content='가갸날을 자랑하겠습니다.'), Document(id='29604c36-ff2c-4e39-9d6f-8c6f144a12ef', metadata={'emotion': {'감동/감탄': 0.8, '고마움': 0.8, '기쁨': 0.8, '기대감': 0.4, '행복': 0.4, '환영/호의': 0.2, '존경': 0.2, '비장함': 0.2, '아껴주는': 0.2, '즐거움/신남': 0.2, '깨달음': 0.2, '놀람': 0.2, '안심/신뢰': 0.2}, 'poet': '김소월'}, page_content='오오 은혜여, 살아있는 몸에는 넘치는 은혜여'), Document(id='d76edcb9-87d2-46c5-a37f-479bed21bbe9', metadata={'emotion': {'흐뭇함(귀여움/예쁨)': 1.0, '환영/호의': 

In [23]:
def extract_poem(text: str) -> str: 
    """ text에서 "### 시:" 이후 나오는 부분만 추출하고, 그 다음 섹션 마커(예: ### 전체:)가 나오면 그 앞까지만 반환. """ 
    start_marker = "### 시:" 
    end_markers = ["### 전체:", "### 해설:", "### 요약:"] # 필요한 경우 확장 가능 
    if start_marker not in text: return "[시를 찾을 수 없습니다]" # 1) 시 시작 부분만 추출 
    poem_section = text.split(start_marker, 1)[1].strip() # 2) 끝 마커가 있으면 그 앞까지만 자르기 
    for end_marker in end_markers: 
        if end_marker in poem_section: 
            poem_section = poem_section.split(end_marker, 1)[0].strip() 
            break # 가장 먼저 등장한 마커까지만 사용 
        return poem_section

In [24]:
extracted_poem = extract_poem(poem)
print(extracted_poem)

시 제목: 아침 미소

미풍에 춤추는 꽃잎들이 웃음소리 내어라
봄바람 타고 오는 설렘이 가슴 가득 차오르니
아침 해가 솟아오르는 창가에 기대어 서서
희망이란 이름의 씨앗을 마음 밭에 심는다

그리움마저 곱게 다듬어진 이 새벽녘
별빛 조각들이 눈물처럼 반짝이며 내려와
살랑이는 산들바람 속에 묻혀버린 아픔도
새로운 시작 앞에서 조용히 사라져간다

청년 같은 열정으로 오늘 하루를 껴안고
푸르른 하늘 바라보며 깊은 숨결 들이킨다
삶이라는 바다 위에 띄우는 작은 배 한 척
꿈과 현실 사이 어딘가를 향해 나아간다

아아, 살아있다는 것 자체가 이토록 소중하니
매 순간마다 새롭게 피어나는 기쁨이여
어제의 상처들은 모두 추억이 되어가고
내일의 희망은 더욱 선명하게 다가온다

가슴속 깊이 새겨진 그리움의 흔적들
모두 따뜻한 햇살 아래 녹아내리며
오늘도 또 내일도 계속될 우리의 이야기
그렇게 삶이라는 무대 위로 걸어 나간다
